In [1]:
import numpy as np
import typing as tp
from random import randint
from cvxopt import matrix, solvers

class MySVM:
    def __init__(self) -> None:
        '''Some words about:
        self._w: array of weights
        self._b: free member
        self._c: free member for poly kernel
        self._d: degree for poly kernel
        self._kernel: our transformation for dual fits
        self._x_train: train x
        self._y_train: train y
        
        '''
        self._w: np.ndarray = np.array([])
        self._b: float = 0
        self._c: float = 1
        self._d: int = 3
        self._kernel: tp.Callable
        self._x_train: np.ndarray = np.array([])
        self._y_train: np.ndarray = np.array([])
 
    def predict(self, x: np.ndarray):
        """ Predict an 0/1 class for samples
        Let’s say n is number of samples , d is number of features.
        Raises RuntimeError if classifier was not fitted yet
        params:
        x: np.array with shape (n, d)
        returns: np.array of length n: predicted class 0/1 for each sample
        """
        if self._x_train.tolist():
            if len(x.shape) == 1:
                x = x[np.newaxis]
            ans = []
            for v in x:
                alph = np.array(self._w)
                ans.append(np.heaviside(sum([val * self._y_train[key][0] * self._kernel(self._x_train[key], v) for key, val in enumerate(alph)])[0], 1))
            return np.array(ans)
        else:
            return np.heaviside(x @ self._w + self._b, 1)
 
    def fit_hard_svm_with_qp(self, x: np.ndarray, y: np.ndarray) -> None:
        """ Train classifier by solving a prime quadratic optimization problem
        Raises RuntimeError if samples are not linearly separable
        params:
        x: np.array with shape (n, d)
        y: np.array with shape (n) - 0/1
        """
        y[y == 0] = -1
        X = np.column_stack((x, np.ones(x.shape[0])))
        n, d = X.shape
        
        P = matrix(np.eye(d))
        G = matrix((-y * np.ones_like(X) * X).astype(float).T.tolist())
        h = matrix(np.full((n), -1).astype(float))
        q = matrix(np.zeros((d)).astype(float))
        
        solv = solvers.qp(P, q, G, h)
        self._w = np.array(solv['x'][:d - 1])
        self._b = solv['x'][d - 1]
        y[y == -1] = 0
 
    def fit_hard_svm_with_qp_dual(self, x: np.ndarray, y: np.ndarray) -> None:
        """ Train classifier by solving a dual quadratic optimization problem
        Raises RuntimeError if samples are not linearly separable
        params:
        x: np.array with shape (n, d)
        y: np.array with shape (n) - 0/1
        """
 
        def kernel(x1, x2):
            return np.dot(x1, x2)
 
        y[y == 0] = -1
        n, d = x.shape
        P = matrix(np.array([[kernel(x1, x2) * y[ind1][0] * y[ind2][0] for ind2, x2 in enumerate(x)] for ind1, x1 in
                             enumerate(x)]))
        q = matrix(np.ones((n, 1)).astype(float))
        G = matrix(np.diag(np.full(n, -1.)))
        h = matrix(np.zeros((n, 1)).astype(float))
        A = matrix(y.T.astype(float))
        b = matrix(0.0)
 
        solv = np.array(solvers.qp(P, q, G, h, A, b)['x'])
    
        self._w = np.array(solv)
        self._kernel = kernel
        self._x_train = np.array(x)
        self._y_train = np.array(y)
        y[y == -1] = 0
 
    def fit_soft_svm_with_qp(self, x: np.ndarray, y: np.ndarray, C: float=1.) -> None:
        """ Train classifier by solving a prime quadratic optimization problem
        0.5 || w ||^2 + C sum(ksi_i)
        params:
        x: np.array with shape (n, d)
        y: np.array with shape (n) - 0/1
        C: float
        """
        y[y == 0] = -1
        X = np.column_stack((x, np.ones(x.shape[0])))
        n, d = X.shape
 
        P = matrix(np.diag([1.] * d + [0.] * n))
        G = matrix(np.concatenate([-np.column_stack(((y * np.ones_like(X) * X), np.eye(n))),
                                np.column_stack((np.zeros((n, d)), -np.eye(n)))]).T.tolist())
        h = matrix(np.concatenate([np.full((n), -1.), np.zeros((n))]))
        q = matrix(np.concatenate([np.zeros((d)), np.full((n), C)]).astype(float))
 
        solv = solvers.qp(P, q, G, h)
        self._w = np.array(solv['x'][:d-1])
        self._b = np.array(solv['x'][d-1])
        y[y == -1] = 0
 
    def fit_soft_svm_with_qp_dual(self, x: np.ndarray, y: np.ndarray, C: float=1.) -> None:
        """ Train classifier by solving a dual quadratic optimization problem
        params:
        x: np.array with shape (n, d)
        y: np.array with shape (n) - 0/1
        C: float
        """
 
        def kernel(x1, x2):
            return np.dot(x1, x2)
 
        y[y == 0] = -1
        n, d = x.shape
        P = matrix(np.array([[kernel(x1, x2) * y[ind1][0] * y[ind2][0] for ind2, x2 in enumerate(x)] for ind1, x1 in
                             enumerate(x)]))
        q = matrix(np.ones((n, 1)).astype(float))
        G = matrix(np.concatenate([np.diag(np.ones(n)), np.diag(np.full(n, -1.))]))
        h = matrix(np.concatenate([np.full((n, 1), C), np.zeros((n, 1))]).astype(float))
        A = matrix(y.T.astype(float))
        b = matrix(0.0)
 
        solv = np.array(solvers.qp(P, q, G, h, A, b)['x'])
    
        self._w = np.array(solv)
        self._kernel = kernel
        self._x_train = np.array(x)
        self._y_train = np.array(y)
        y[y == -1] = 0
 
    def fit_soft_svm_with_sgd(self, x: np.ndarray, y: np.ndarray, C: float=1.) -> None:
        """ Train classifier with SGD
        params:
        X: np.array with shape (n, d)
        y: np.array with shape (n) - 0/1
        C: float
        """
        
        def kernel(x1, x2):
            return np.dot(x1, x2)
        
        y[y == 0] = -1
        x = np.column_stack((x, np.ones(x.shape[0])))
        n, d = x.shape
        lambd = 1.0
        theta = np.zeros((n + 1, d))
        w = np.zeros(x.shape)
 
        for t in range(n):
            w[t] = theta[t] * (1 / (lambd * (t + 1)))
            i = randint(0, n - 1)
            if y[i][0] * kernel(w[t], x[i]) < 1:
                theta[t + 1] = theta[t] + C * y[i][0] * x[i]
            else:
                theta[t + 1] = theta[t]
        w = np.sum(w, axis=0)
        self._w = np.array(w[:d-1])
        self._b = np.array(w[d-1])
        y[y == -1] = 0
 
    def fit_soft_svm_with_poly_kernel(self, x: np.ndarray, y: np.ndarray,
                                      C: float=1., c: float=1., d: int=3) -> None:
        """ Train classifier with poly kernel
        K(a, b) = (<a, b> + c)^d
        params:
        x: np.array with shape (n, d)
        y: np.array with shape (n) - 0/1
        C: float
        c: float
        d: int
        """
 
        def kernel(x1, x2, c=c, d=d):
            return (np.dot(x1, x2) + c) ** d
 
        y[y == 0] = -1
        n, dim = x.shape
        P = matrix(np.array(
            [[K(x1, x2, c, d) * y[ind1][0] * y[ind2][0] for ind2, x2 in enumerate(x)]
             for ind1, x1 in enumerate(x)]))
        q = matrix(np.ones((n, 1)).astype(float))
        G = matrix(np.concatenate([np.diag(np.ones(n)), np.diag(np.full(n, -1.))]))
        h = matrix(np.concatenate([np.full((n, 1), C), np.zeros((n, 1))]).astype(float))
 
        A = matrix(y.T.astype(float))
        b = matrix(0.0)
 
        solv = np.array(solvers.qp(P, q, G, h, A, b)['x'])
        self._w = np.array(solv)
        self._kernel = kernel
        self._x_train = np.array(x)
        self._y_train = np.array(y)
        y[y == -1] = 0
        

In [3]:
a=MySVM()

In [4]:
a